In [1]:
%load_ext autoreload
%autoreload 2`

In [2]:
# import hvplot.pandas
# import holoviews as hv
# import panel as pn
import pandas as pd
import numpy as np
# import param as pm
import random
import math
# pn.extension(sizing_mode="stretch_width")
# hv.extension("bokeh")
import os, sys
sys.path.append('/home/bk/LTF-base/aaa-tokenomics')
# pn.extension('plotly')

In [3]:
from AAA.sim_handler import SimHandler

In [4]:
s = SimHandler()

In [5]:
df = s.run()

instantiate_Users
instantiate_Bonds
Iteration------------------------ 0
[User-1: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 0, 'ETH': 1},
 User-2: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 0, 'ETH': 1},
 User-3: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 0, 'ETH': 1},
 User-4: Bal={'DAI': 1000, 'AHM': 0, 'sAHM': 0, 'ETH': 1}]
Iteration------------------------ 1
bond_price: 1.0
bond_price: 1.0
Total AHM supply: 2.0
bond_price: 1.0
Total AHM supply: 2.0
bond_price: 1.0
Total AHM supply: 4.0
bond_price: 1.0
Total AHM supply: 4.0
bond_price: 1.0
Total AHM supply: 6.0
bond_price: 1.0
Total AHM supply: 6.0
bond_price: 1.0
Tresury
{'DAI': 400, 'AHM': 0}
DAO
{'DAI': 0, 'AHM': 8.0}
BondInfo
{1: {'payout': 100.0, 'vesting': 5, 'lastBlock': 1, 'pricePaid': 100},
 2: {'payout': 100.0, 'vesting': 5, 'lastBlock': 1, 'pricePaid': 100},
 3: {'payout': 100.0, 'vesting': 5, 'lastBlock': 1, 'pricePaid': 100},
 4: {'payout': 100.0, 'vesting': 5, 'lastBlock': 1, 'pricePaid': 100}}
Total Debt
400.0
[Bond-DAI]
[User-1: Bal={'DA

In [9]:
df

,0,1,2
0,0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
1,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
2,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
3,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
4,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
5,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
6,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
7,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
8,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"
9,400.0,"{'DAI': 400, 'AHM': 0}","{'DAI': 0, 'AHM': 8.0}"


In [6]:
s.bonds[0].adjustment

{'add': False, 'rate': 0.1, 'target': 1, 'buffer': 5, 'lastBlock': 0}

In [7]:
# from AAA.bond import Bond

# b= Bond()

# b.deposit(s.users[0],10)

# b

In [8]:
s.bonds[0].bond_price()

Total AHM supply: 420.1204
bond_price: 1.0


1.0